In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [3]:
######################DONT NEED THESE LINES, USE PATH IN GCP DIRS######################################
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = "/content/drive/MyDrive/Documents/imagenette2/train" #########NEED TO CHANGE THIS PATH ACC TO GCP DIRS
root_dir = data_dir
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader

trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

#natural_img_dataset = datasets.ImageFolder(root = root_dir,transform = image_transforms["train"])
#natural_img_dataset

['n01440764', 'n02102040', 'n02979186', 'n03000684', 'n03028079', 'n03394916', 'n03417042', 'n03425413', 'n03445777', 'n03888257']


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet50(pretrained=True)
#print(model)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [6]:
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
import time

t1 = time.time()
epochs = 1
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
  for inputs, labels in trainloader:
    steps += 1
    print(steps)
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    logps = model.forward(inputs)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print("trainloader done") 
  #if steps % print_every == 0:
  test_loss = 0
  accuracy = 0
  model.eval()
  with torch.no_grad():

    for inputs, labels in testloader:
      inputs, labels = inputs.to(device),labels.to(device)
      logps = model.forward(inputs)
      batch_loss = criterion(logps, labels)
      test_loss += batch_loss.item()
      
      ps = torch.exp(logps)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
  train_losses.append(running_loss/len(trainloader))
  test_losses.append(test_loss/len(testloader))                    
  print(f"Epoch {epoch+1}/{epochs}.. "
  f"Train loss: {running_loss/print_every:.3f}.. "
  f"Test loss: {test_loss/len(testloader):.3f}.. "
  f"Test accuracy: {accuracy/len(testloader):.3f}")
  running_loss = 0
  model.train()
print("Saving Model")
torch.save(model, '/content/drive/MyDrive/Documents/imagenette2/aerialmodel.pth') #########NEED TO CHANGE THIS PATH ACC TO GCP DIRS

print("Training time per epoch is {} seconds".format(time.time() - t1))

Hi
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
trainloader done
Epoch 1/1.. Train loss: 2.555.. Test loss: 0.055.. Test accuracy: 0.979
Saving Model
Training time per epoch is 5314.126418352127 seconds


In [30]:
import time
from torch.autograd import Variable
data_dir = "/content/drive/MyDrive/Documents/imagenette2/train" #########NEED TO CHANGE THIS PATH ACC TO GCP DIRS
test_transforms = transforms.Compose([transforms.Resize((224, 224)),
        transforms.ToTensor()
                                     ])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('/content/drive/MyDrive/Documents/imagenette2/aerialmodel.pth') #########NEED TO CHANGE THIS PATH ACC TO GCP DIRS
model.eval()

def predict_image(image):
  image_tensor = test_transforms(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  input = Variable(image_tensor)
  input = input.to(device)
  output = model(input)
  index = output.data.cpu().numpy().argmax()
  return index

def get_random_images(num):
  data = datasets.ImageFolder(data_dir, transform=test_transforms)
  classes = data.classes
  indices = list(range(len(data)))
  np.random.shuffle(indices)
  idx = indices[:num]
  from torch.utils.data.sampler import SubsetRandomSampler
  sampler = SubsetRandomSampler(idx)
  loader = torch.utils.data.DataLoader(data, 
                  sampler=sampler, batch_size=num)
  dataiter = iter(loader)
  images, labels = dataiter.next()
  return images, labels

In [ ]:
t1 = time.time()
to_pil = transforms.ToPILImage()
images, labels = get_random_images(1000)
#fig=plt.figure(figsize=(10,10))
for ii in range(len(images)):
    print(ii+1)
    image = to_pil(images[ii])
    index = predict_image(image)
    #sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    #sub.set_title(str(trainloader.dataset.classes[index]) + ":" + str(res))
    #plt.axis('off')
    ##plt.imshow(image)
#plt.show()
print("Inference time is {} seconds".format(time.time() - t1))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
#########################################################################################################
####################################    VGG16 on Pytorch   ##############################################
#########################################################################################################
from torchvision import transforms

# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.vgg16(pretrained=True)
#print(model)

for param in model.parameters():
    param.requires_grad = False

model.classifier[6] = nn.Sequential(
                      nn.Linear(4096, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, 10),                   
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [24]:
t1 = time.time()
# Early stopping details
n_epochs_stop = 1
min_val_loss = np.Inf
epochs_no_improve = 0
checkpoint_path = "/content/drive/MyDrive/Documents/imagenette2/vgg16model.pth"
# Main loop
for epoch in range(1):
  # Initialize validation loss for epoch
  val_loss = 0
  
  # Training loop
  for data, targets in trainloader:
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()
    
# Validation loop
for data, targets in testloader:
  # Generate predictions 
  out = model(data)
  # Calculate loss
  loss = criterion(out, targets)
  val_loss += loss

# Average validation loss
val_loss = val_loss / len(trainloader)

# If the validation loss is at a minimum
if val_loss < min_val_loss:
  # Save the model
  torch.save(model, checkpoint_path)
  epochs_no_improve = 0
  min_val_loss = val_loss
  
else:
  epochs_no_improve += 1
  # Check early stopping condition
  if epochs_no_improve == n_epochs_stop:
    print('Early stopping!')
    
    # Load in the best model
    model = torch.load(checkpoint_path)
print("Training time per epoch is {} seconds".format(time.time() - t1))

In [33]:
model = torch.load(checkpoint_path)
def predict_image(image):
  image_tensor = test_transforms(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  input = Variable(image_tensor)
  input = input.to(device)
  output = model(input)
  index = output.data.cpu().numpy().argmax()
  return index

def get_random_images(num):
  data = datasets.ImageFolder(data_dir, transform=test_transforms)
  classes = data.classes
  indices = list(range(len(data)))
  np.random.shuffle(indices)
  idx = indices[:num]
  from torch.utils.data.sampler import SubsetRandomSampler
  sampler = SubsetRandomSampler(idx)
  loader = torch.utils.data.DataLoader(data, 
                  sampler=sampler, batch_size=num)
  dataiter = iter(loader)
  images, labels = dataiter.next()
  return images, labels

t1 = time.time()
to_pil = transforms.ToPILImage()
images, labels = get_random_images(1000)
#fig=plt.figure(figsize=(10,10))
for ii in range(len(images)):
    print(ii+1)
    image = to_pil(images[ii])
    index = predict_image(image)
    #sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    #sub.set_title(str(trainloader.dataset.classes[index]) + ":" + str(res))
    #plt.axis('off')
    ##plt.imshow(image)
#plt.show()
print("Inference time is {} seconds".format(time.time() - t1))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
